<a href="https://colab.research.google.com/github/tuky-g/LLM-QAproject/blob/main/1_Vectordatabase_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importar librerías


In [1]:
!pip install jupyter_bokeh

In [2]:
!pip install pypdf

In [3]:
!pip install langchain_community

In [4]:
! pip install chromadb

In [ ]:
!pip install tiktoken

In [ ]:
!pip install --upgrade --quiet  langchain sentence_transformers

In [ ]:
!pip install langchain_huggingface

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
import getpass
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings


In [ ]:
import os
import openai
import sys
sys.path.append('../..')
import panel as pn
import param

# import panel as pn  # GUI
# pn.extension()



In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
HF_API_KEY = os.environ.get('HF_TOKEN')

# Load pdfs

In [ ]:
#connect to folder in drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path_pdf='/content/drive/MyDrive/LLM-QAproject/pdfs/'

In [ ]:
# create loader for each pdf file
list_files = os.listdir(path_pdf)
loaders = []
for file in list_files:
  path_file=os.path.join(path_pdf, file)
  loader = PyPDFLoader(path_file)
  loaders.append(loader)

In [ ]:
# Load Pdfs into docs
docs = []
for loader in loaders:
  print(loader.source)
  docs.extend(loader.load())

In [ ]:
len(docs)

# split

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [ ]:
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

# Embeddings

In [ ]:
# embedding = OpenAIEmbeddings()

# embedding = HuggingFaceInferenceAPIEmbeddings(
#     api_key=inference_api_key, model_name="sentence-transformers/all-MiniLM-l6-v2"
# )

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
persist_directory='/content/drive/MyDrive/LLM-QAproject/docs/chroma4/'

In [ ]:
!rm -rf /content/drive/MyDrive/LLM-QAproject/docs/chroma4/  # remove old database files if any
# !mkdir persist_directory

In [ ]:

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())